In [ ]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="CALLER")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [ ]:
import socket
import sys
import requests
#import requests_oauthlib
import json

def send_tweets_to_spark(full_tweet, tcp_connection):
    try:
        print ("------------------------------------------")
        print("Text: " + full_tweet)
        print(tcp_connection.send(full_tweet.encode('utf-8')))
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

TCP_IP = "localhost"
TCP_PORT = 10004
conn = None


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)


print("Waiting for TCP connection...")
conn, addr = s.accept()

In [ ]:
import os
import tweepy as tw
import time
import random

#Twitter max 100 tweets

consumer_key= 'Gz4fafovYrgtQmAfIOu7a4SgY'
consumer_secret= 'u5X3w7a9NrJT6GIe4sb2hAw0zzt0OyDVdtwfHVWsaAvql62IfV'
access_token= '2592398337-Cxns81v6BWzTAt43uxAN1wh0jlrsOdFP8LHn2cP'
access_token_secret= 'VOQh68hAcLhXEncddElNiMyuAg7Kx5T9GviBnjyHx6rvS'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

for item in api.search_tweets(q="cryptocurrencies", count=100, lang="en"):
    phrase_2_send = item.text + " " + str(item.created_at)
    send_tweets_to_spark(phrase_2_send+"\n", conn)
    time.sleep(random.randint(1,5))
    

#Gdelt max 250 news

from gdeltdoc import GdeltDoc, Filters

f = Filters(
    keyword = "cryptocurrencies",
    start_date = "2020-05-10",
    end_date = "2021-05-11"
)

gd = GdeltDoc()

gd.article_search(f)

for i in gd.article_search(f).values:
    phrase_2_send = i[2] + " " + i[3]
    send_tweets_to_spark(phrase_2_send+"\n", conn)
    time.sleep(random.randint(1,5))

# Scrapeo set 50 pages, cada pagina tiene muchas news

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

from bs4 import BeautifulSoup
i = 1
import time
while True:
    url = "https://coinmarketcal.com/en/news?page={}".format(i)
    source_code = requests.get(url, headers=headers,timeout=5).text
    soup = BeautifulSoup(source_code,"html5lib")
    for news,fecha in zip(soup.body.find_all('h5',class_="card__title"), soup.body.find_all('h5',class_="card__date")):
        phrase_2_send = news.text + " " + fecha.text
        send_tweets_to_spark(phrase_2_send+"\n", conn)
        time.sleep(random.randint(1,5))
    i = i + 1
    if i==50:
        break